In [1]:
import config.env

In [2]:
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loaders = [
 TextLoader("data/荣耀Magic V Flip.txt","utf-8"),
    TextLoader("data/Xiaomi MIX Flip.txt","utf-8"),
    TextLoader("data/Find N3 Flip.txt","utf-8"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
docs = text_splitter.split_documents(docs)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

In [3]:
len(docs)

3

In [4]:
docs

[Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备注：可使用的内存容量小于此值，因为手机软件占用部分空间。\n\n## 拍照\n\n后置

检索较大的信息块但嵌入较小的信息块可能很有用。这允许嵌入尽可能接近地捕获语义，但可以将尽可能多的上下文传递到下游。请注意，这就是ParentDocumentRetriever的作用

将区分矢量存储和文档存储，矢量存储对（子）文档的嵌入进行索引，文档存储存储“父”文档并将它们与标识符相关联

In [5]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever

# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

doc_ids = [str(uuid.uuid4()) for _ in docs]

拆分原始文档来生成“子”文档。请注意，我们将文档标识符存储在相应Document对象的metadata中。

In [6]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [7]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [8]:
doc_ids

['194a6446-3312-4d0d-b0c2-7570a2fbdf2b',
 '7bc55fa8-f399-490c-8191-7145ac26b59b',
 'e8c2c379-89c1-4349-8d10-5b767db1acc4']

In [9]:
docs

[Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备注：可使用的内存容量小于此值，因为手机软件占用部分空间。\n\n## 拍照\n\n后置

In [10]:
list(zip(doc_ids, docs))

[('194a6446-3312-4d0d-b0c2-7570a2fbdf2b',
  Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备

In [11]:
len(list(zip(doc_ids, docs)))

3

In [12]:
retriever.vectorstore.similarity_search("荣耀Magic V Flip的颜色")

[Document(metadata={'doc_id': '194a6446-3312-4d0d-b0c2-7570a2fbdf2b', 'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕'),
 Document(metadata={'doc_id': 'e8c2c379-89c1-4349-8d10-5b767db1acc4', 'source': 'data/Find N3 Flip.txt'}, page_content='# Find N3 Flip\n\n## 颜色\n\n月光缪斯 | 薄雾玫瑰 | 镜中之夜\n\n入网型号：PHT110\n\n## 尺寸与重量¹\n\n高\n展开：166.4 毫米，折叠：85.5 毫米\n宽\n展开：75.8 毫米，折叠：75.8 毫米\n厚\n展开：7.79 毫米，折叠：16.45 毫米\n重量\n198 克\n\n## 存储²\n\nRAM 容量 + ROM 容量\n12GB + 256GB，12GB + 512GB\nRAM 规格\nLPDDR5X\nROM 规格\nUFS 4.0\n扩展存储\n不支持\nOTG 数据传输\n支持\n\n## 显示'),
 Document(metadata={'doc_id': '7bc55fa8-f399-490c-8191-7145ac26b59b', 'source': 'data/Xiaomi MIX Flip.txt'}, page_content='# Xiaomi MIX Flip\n\n## 概况\n\n全尺寸多功能超大外屏\n第三代骁龙.8 移动平台|动态性能调度 4.0\n4780mAh 小米金沙江电池\n3D立体式VC散热系统\n67W 小米澎湃有线秒充\n

In [13]:
retriever.invoke("荣耀Magic V Flip的颜色")

[Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备注：可使用的内存容量小于此值，因为手机软件占用部分空间。\n\n## 拍照\n\n后置

In [14]:
retriever.vectorstore.similarity_search("荣耀Magic V Flip的屏幕尺寸")

[Document(metadata={'doc_id': '194a6446-3312-4d0d-b0c2-7570a2fbdf2b', 'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕'),
 Document(metadata={'doc_id': '7bc55fa8-f399-490c-8191-7145ac26b59b', 'source': 'data/Xiaomi MIX Flip.txt'}, page_content='# Xiaomi MIX Flip\n\n## 概况\n\n全尺寸多功能超大外屏\n第三代骁龙.8 移动平台|动态性能调度 4.0\n4780mAh 小米金沙江电池\n3D立体式VC散热系统\n67W 小米澎湃有线秒充\n徕卡光学 Summilux 镜头\n徕卡主摄\n徕卡浮动长焦\n小米龙铠架构|小米龙晶玻璃\n专业1.5K 超视感屏|全等深微曲屏\n立体声双扬声器\n\n## 外观尺寸\n\n长度:167.5mm\n宽度:74.02mm\n厚度:展开-上/下侧7.8mm(小米凤羽纤维版)、上侧7.8mm(玻璃)、下侧7.6mm(玻璃)\n折叠-16.19mm(小米凤羽纤维版)、15.99mm(玻璃)\n重量(不含膜):192g(玻璃)、190g(小米凤羽纤维版)\n“以上数据为小米实验室测试数据，依据行业内测量方式不同，实际结果可能略有差异。\n\n## 内存容量'),
 Document(metadata={'doc_id': 'e8c2c379-89c1-4349-8d10-5b767db1acc4', 'source': 'data/Find N3 Flip.txt'}, page_content='# Find N3 Fl

In [15]:
retriever.invoke("荣耀Magic V Flip的屏幕尺寸")

[Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备注：可使用的内存容量小于此值，因为手机软件占用部分空间。\n\n## 拍照\n\n后置

In [16]:
retriever.vectorstore.similarity_search("Find N3 Flip-显示-尺寸")

[Document(metadata={'doc_id': 'e8c2c379-89c1-4349-8d10-5b767db1acc4', 'source': 'data/Find N3 Flip.txt'}, page_content='# Find N3 Flip\n\n## 颜色\n\n月光缪斯 | 薄雾玫瑰 | 镜中之夜\n\n入网型号：PHT110\n\n## 尺寸与重量¹\n\n高\n展开：166.4 毫米，折叠：85.5 毫米\n宽\n展开：75.8 毫米，折叠：75.8 毫米\n厚\n展开：7.79 毫米，折叠：16.45 毫米\n重量\n198 克\n\n## 存储²\n\nRAM 容量 + ROM 容量\n12GB + 256GB，12GB + 512GB\nRAM 规格\nLPDDR5X\nROM 规格\nUFS 4.0\n扩展存储\n不支持\nOTG 数据传输\n支持\n\n## 显示'),
 Document(metadata={'doc_id': '194a6446-3312-4d0d-b0c2-7570a2fbdf2b', 'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕'),
 Document(metadata={'doc_id': '7bc55fa8-f399-490c-8191-7145ac26b59b', 'source': 'data/Xiaomi MIX Flip.txt'}, page_content='# Xiaomi MIX Flip\n\n## 概况\n\n全尺寸多功能超大外屏\n第三代骁龙.8 移动平台|动态性能调度 4.0\n4780mAh 小米金沙江电池\n3D立体式VC散热系统\n67W 小米澎湃有线秒充\n

In [17]:
retriever.invoke("Find N3 Flip-显示-尺寸")

[Document(metadata={'source': 'data/Find N3 Flip.txt'}, page_content='# Find N3 Flip\n\n## 颜色\n\n月光缪斯 | 薄雾玫瑰 | 镜中之夜\n\n入网型号：PHT110\n\n## 尺寸与重量¹\n\n高\n展开：166.4 毫米，折叠：85.5 毫米\n宽\n展开：75.8 毫米，折叠：75.8 毫米\n厚\n展开：7.79 毫米，折叠：16.45 毫米\n重量\n198 克\n\n## 存储²\n\nRAM 容量 + ROM 容量\n12GB + 256GB，12GB + 512GB\nRAM 规格\nLPDDR5X\nROM 规格\nUFS 4.0\n扩展存储\n不支持\nOTG 数据传输\n支持\n\n## 显示\n\n尺寸\n内屏：6.80 英寸（对角线）\n外屏：3.26 英寸（对角线）\n分辨率\n内屏：2520 × 1080 像素，403 PPI\n外屏：720 × 382 像素，250 PPI\n色彩\n内屏：10.7 亿色，支持广色域（P3）\n外屏：1670 万色，支持广色域（P3）\n亮度\n内屏：\n1600 尼特峰值亮度\n1200 尼特峰值亮度（HBM 典型）\n外屏：\n900 尼特峰值亮度\n800 尼特峰值亮度（HBM 典型）\n类型\n内屏：\nAMOLED 柔性屏\n1～120Hz 自适应刷新率\n最高支持 240Hz 触控采样率\n覆盖肖特 UTG 玻璃\n外屏：\nAMOLED 柔性屏\n最高支持 60Hz 刷新率\n最高支持 120Hz 触控采样率\n覆盖康宁® 第七代大猩猩® 玻璃\n\n## 摄像头\n\n后置\n5000 万像素主摄像头：ƒ/1.8，等效焦距 24 毫米，六镜式镜头，支持 AF，采用闭环对焦马达，支持 OIS\n3200 万像素长焦摄像头：ƒ/2.0，等效焦距 47 毫米，六镜式镜头，支持 AF\n4800 万像素超广角摄像头：ƒ/2.2，FOV 114°，等效焦距 14 毫米，六镜式镜头，支持 AF，支持 4 厘米微距拍摄\n照片拍摄支持最高 2 倍光学变焦，20 倍数码变焦\n支持：夜景，视频，照片，人像，专业，全景，电影，慢动作，长曝光，延时摄影，萌拍，AI 证件照，超级文本，XPAN，高像素\n照片格

## 将摘要与文档关联起来

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.environ["OPENAI_MODEL_ID"])

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("总结以下文件:\n\n{doc}")
    | llm
    | StrOutputParser()
)

In [19]:
summaries = chain.batch(docs, {"max_concurrency": 5})
summaries 

['## 荣耀Magic V Flip 概述\n\n### 颜色选择\n- 香槟粉\n- 山茶白\n- 鸢尾黑\n- 高定款\n\n### 尺寸与重量\n- 展开态尺寸：167.3 mm x 75.6 mm x 7.15 mm\n- 折叠态尺寸：86.5 mm x 75.6 mm x 14.89 mm\n- 重量：约193克\n\n### 屏幕\n- 外屏：4.0英寸，分辨率1200x1092像素\n- 内屏：6.8英寸，分辨率2520x1080像素\n- 类型：OLED，支持DCI-P3广色域\n- 护眼技术：高频PWM调光、护眼模式\n- 多点触控（最多支持10点）\n\n### 处理器\n- CPU：第一代骁龙8+，八核（最高3.0GHz）\n- GPU：Adreno 730\n\n### 操作系统\n- MagicOS 8.0（基于Android 14）\n\n### 存储选项\n- 12GB RAM + 256GB/512GB/1TB 全网通\n- 16GB RAM + 1TB（高定款）\n\n### 拍照功能\n- 后置双摄：5000万主摄 + 1200万超广角微距\n- 前置摄像头：5000万像素\n- 支持4K视频录制和多种拍摄模式（如夜景、AI摄影等）\n\n### 电池\n- 容量：4800mAh\n- 支持66W快充\n\n### 网络与连接\n- 支持多种5G/4G网络制式\n- WLAN、蓝牙5.3、NFC、OTG等多种连接选项\n\n### 传感器\n- 包含重力、指纹、陀螺仪、指南针、环境光等传感器\n\n### 价格\n- 12GB+256GB: RMB 4999\n- 12GB+512GB: RMB 5499\n- 12GB+1TB: RMB 5999\n\n### 包装清单\n- 包含手机、充电器、数据线、保护壳、保护膜等配件\n\n此设备兼具高性能、优雅外观和多样的功能，适合追求时尚与技术的用户。',
 '# 小米MIX Flip 概况总结\n\n## 产品特性\n- **外观设计**: 配备全尺寸多功能超大外屏，折叠后厚度16.19mm，重量192g（玻璃版）。\n- **电池与充电**: 4780mAh电池，支持67W快充。\n- **处理器**: 搭载第三代骁龙8移动平台，采用台积电4nm工艺。\n- **内存与存储**:

对向量存储中的摘要进行索引，并在文档存储中保留原始文档

In [20]:
from langchain_core.documents import Document
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=OpenAIEmbeddings())
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [21]:
doc_ids

['4ba5b3e8-5055-4f74-a996-ab414b193eb2',
 '4934d315-5848-4d1a-b825-a6df293b9375',
 '27d43510-b734-4293-9be7-832ca10ff79a']

In [ ]:
# # We can also add the original chunks to the vectorstore if we so want
# for i, doc in enumerate(docs):
#     doc.metadata[id_key] = doc_ids[i]
# retriever.vectorstore.add_documents(docs)

In [22]:
sub_docs = retriever.vectorstore.similarity_search("荣耀Magic V Flip的颜色")
sub_docs

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={'doc_id': '4ba5b3e8-5055-4f74-a996-ab414b193eb2'}, page_content='## 荣耀Magic V Flip 概述\n\n### 颜色选择\n- 香槟粉\n- 山茶白\n- 鸢尾黑\n- 高定款\n\n### 尺寸与重量\n- 展开态尺寸：167.3 mm x 75.6 mm x 7.15 mm\n- 折叠态尺寸：86.5 mm x 75.6 mm x 14.89 mm\n- 重量：约193克\n\n### 屏幕\n- 外屏：4.0英寸，分辨率1200x1092像素\n- 内屏：6.8英寸，分辨率2520x1080像素\n- 类型：OLED，支持DCI-P3广色域\n- 护眼技术：高频PWM调光、护眼模式\n- 多点触控（最多支持10点）\n\n### 处理器\n- CPU：第一代骁龙8+，八核（最高3.0GHz）\n- GPU：Adreno 730\n\n### 操作系统\n- MagicOS 8.0（基于Android 14）\n\n### 存储选项\n- 12GB RAM + 256GB/512GB/1TB 全网通\n- 16GB RAM + 1TB（高定款）\n\n### 拍照功能\n- 后置双摄：5000万主摄 + 1200万超广角微距\n- 前置摄像头：5000万像素\n- 支持4K视频录制和多种拍摄模式（如夜景、AI摄影等）\n\n### 电池\n- 容量：4800mAh\n- 支持66W快充\n\n### 网络与连接\n- 支持多种5G/4G网络制式\n- WLAN、蓝牙5.3、NFC、OTG等多种连接选项\n\n### 传感器\n- 包含重力、指纹、陀螺仪、指南针、环境光等传感器\n\n### 价格\n- 12GB+256GB: RMB 4999\n- 12GB+512GB: RMB 5499\n- 12GB+1TB: RMB 5999\n\n### 包装清单\n- 包含手机、充电器、数据线、保护壳、保护膜等配件\n\n此设备兼具高性能、优雅外观和多样的功能，适合追求时尚与技术的用户。'),
 Document(metadata={'doc_id': '27d43510-b734-4293-9be7-832ca10ff79a'}, page_conten

In [24]:
retrieved_docs = retriever.invoke("荣耀Magic V Flip的颜色")
retrieved_docs

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(metadata={'source': 'data/荣耀Magic V Flip.txt'}, page_content='# 荣耀Magic V Flip\n\n规格参数\n\n## 颜色\n\n香槟粉,\n山茶白,\n鸢尾黑,\n高定款\n\n## 尺寸与重量\n\n长度\n展开态：167.3 mm\n折叠态：86.5 mm\n宽度\n展开态：75.6 mm\n折叠态：75.6 mm\n厚度\n展开态：7.15 mm\n折叠态：14.89 mm\n机身重量\n约193克（含电池）\n备注：实际尺寸/重量依配置、制造工艺、测量方法的不同可能有所差异。\n\n## 屏幕\n\n屏幕尺寸\n外屏：4.0英寸\n内屏：6.8英寸\n备注：显示屏采用圆角设计，按照标准矩形测量内屏屏幕的对角线长度为6.8英寸（实际可视区域略小），外屏屏幕的对角线长度是4.0英寸（实际可视区域略小）。\n屏幕色彩\nDCI-P3广色域\n屏幕类型\nOLED\n屏幕分辨率\n内屏：2520x1080像素\n外屏：1200x1092 像素\n备注：该分辨率对应标准矩形，实际屏幕有效像素略少。\n护眼技术\n内屏：3840Hz超高频PWM调光、护眼模式、类自然光护眼、助眠显示\n外屏：2160Hz高频PWM调光、护眼模式、类自然光护眼、助眠显示\n备注：本产品非医疗器械，不具有治疗功能。\n屏幕触控\n多点触控，最多支持10点触控\n\n## 处理器\n\nCPU型号\n第一代骁龙8+移动平台\nCPU核数\n八核\nCPU主频\n1 × Cortex-X2 3.0GHz + 3 × Cortex-A710 2.5GHz + 4 × Cortex-A510 1.8GHz\n备注：实际运行频率因应用负载智能调整。\nGPU\nAdreno™ 730\n\n## 键盘类型\n\n手势导航（默认）、屏幕内三键导航、悬浮导航\n\n\n## 操作系统\n\nMagicOS 8.0（基于Android 14）\n用户界面\nMagicOS 8.0\n\n## 存储\n\n12GB+256GB 全网通\n12GB+512GB 全网通\n12GB+1TB 全网通\n16GB+1TB 全网通（高定款）\n备注：可使用的内存容量小于此值，因为手机软件占用部分空间。\n\n## 拍照\n\n后置